In [40]:
# bs4 for scraping, requests for requesting, pandas for data-ing
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [41]:
# First website to be scraped
URL1 = 'https://www.nonprofitnewyork.org/join/directory/'
page = requests.get(URL1)

In [42]:
# scrape relevant HTML
soup1 = BeautifulSoup(page.content, 'html.parser')
all_non_profits = soup1.select('.tab-pane > ul > li')

In [43]:
# process scraped HTML and extract website + non profite name
nps = []

for np in all_non_profits:
    a = np.find('a')
    if a:
        nps.append({'website': a['href'], 'name': a.contents[0]})
    else:
        # text of the non linked nps is the second child of the li tag
        nps.append({'name': np.contents[1]})

# Save the data
df1 = pd.DataFrame(nps)
# print(df1.head())
df1.to_csv('1.csv')

In [44]:
# onto the next site, should be a similar process to above
URL2 = 'https://queensbp.org/nonprofits/'
page = requests.get(URL2)
soup2 = BeautifulSoup(page.content, 'html.parser')
all_non_profits2 = soup2.select('#tablepress-1 > tbody > tr')

In [45]:
nps2 = []

for np in all_non_profits2:
    name, websiteCol = np.find_all('td')
    website = websiteCol.find('a')
    row = {'name': name.contents[0]}
    if website:
        row['website'] = website.contents[0]
    nps2.append(row)

df2 = pd.DataFrame(nps2)
df2.to_csv('2.csv')

In [61]:
# 3rd Site - note the large 100000 argument in the url to get all listings on one page
URL3 = 'https://www.nycservice.org/organizations/index.php?sort_mode=title_asc&max_records=100000'
page = requests.get(URL3)
soup3 = BeautifulSoup(page.content, 'html.parser')
# The indexing at the end cuts off the first entry as it is the table header, not an NGO
all_non_profits3 = soup3.select('.data > tr')[1:]

In [77]:
nps3 = []

for np in all_non_profits3:
    nameCol, descriptionCol = np.find_all('td')
    nameA = nameCol.find('a')
    fullInfoPage = 'https://www.nycservice.org' + nameA['href']
    row = {'name': nameA.contents[0], 'description': descriptionCol.contents[0]}
    
    # Each ngo has its own page on the website with more useful info to scrape
    companyPage = requests.get(fullInfoPage)
    tempSoup = BeautifulSoup(companyPage.content, 'html.parser')
    
    website = tempSoup.select('.orginfo > a')
    if(website):
        row['website'] = website[0]['href']
   
    address = tempSoup.select('.address > a')
    if address: 
        addressText = []
        for line in address[0].contents:
            if str(line) != '<br/>':
                addressText.append(line)
        row['address'] = ' '.join(addressText)
    # print(row)
    nps3.append(row)

df3 = pd.DataFrame(nps3)
df3.to_csv('3.csv')    